# Wdrożenie

## Docker

*Mała uwaga na wstępie odnośnie klasy `PolBertFeatureConstructor()` w skrypcie `preprocessing.py`. Niestety z uwagi na problemy jakie miałem z użyciem bibliotek DL, odinstalowałem ze środowiska PyTorcha i jego komponentów, aby nie potrzebnie "doważać" środowisko i potem Dockera.*

Mając już gotowy model, przeprowadzimy proces wdrożenia go do środowiska "produkcyjnego", tzn. mój laptop.

Na wstępie zrzut środowiska.

In [6]:
!pip freeze > requirements.txt

Definiuje schemat stworzenia Dockera - kopiuje do jego środowiska tylko i wyłącznie niezbędne komponenty potrzebne do uruchomienia modelu na produkcji, czyli

1. `requirements.txt` - definicja środowiska
2. `app.py` - aplikacja we Flasku wystawiająca API
3. `production` - folder z modelem wdrażanymi na produkcję
4. `preprocessing` - skrypty do przygotowania tekstu podawanego do API

In [11]:
!cat Dockerfile

FROM python:3.7
WORKDIR /app
COPY requirements.txt /app
RUN pip install -r ./requirements.txt
COPY app.py /app
COPY production /app/production
COPY preprocessing.py /app
CMD ["python", "app.py"]


Stawiamy Dockera zgodnie z definicją jego środowiska wyżej.

*Aby skrócić wynik w konsoli, pozwoliłem mu na zaciągnięcie środowiska z cache'a*

In [12]:
!sudo docker build -t marcin_zadanie_rekrutacyjne:latest .

Sending build context to Docker daemon  862.6MBxt to Docker daemon  557.1kB
Step 1/8 : FROM python:3.7
 ---> 9569e8192573
Step 2/8 : WORKDIR /app
 ---> Using cache
 ---> 05adcb201725
Step 3/8 : COPY requirements.txt /app
 ---> Using cache
 ---> bd4f0356cf81
Step 4/8 : RUN pip install -r ./requirements.txt
 ---> Using cache
 ---> 1ba220a7b086
Step 5/8 : COPY app.py /app
 ---> b231d68743e5
Step 6/8 : COPY production /app/production
 ---> 3248ac46602d
Step 7/8 : COPY preprocessing.py /app
 ---> 5bb2c44dfc80
Step 8/8 : CMD ["python", "app.py"]
 ---> Running in 768c5ee31084
Removing intermediate container 768c5ee31084
 ---> ac24ce7b322d
Successfully built ac24ce7b322d
Successfully tagged marcin_zadanie_rekrutacyjne:latest


Rozmiar zbudowanego w ten sposób Dockera wynosi 1.89 GB (pierwsza linijka)

*Przepraszam za pozostałe obrazy, które niepotrzebnie utrudniają analizę i wydłużają wynik z konsoli*

In [21]:
!sudo docker images

[sudo] password for marcin: REPOSITORY                                       TAG       IMAGE ID       CREATED             SIZE
marcinrzadanie/marcin_zadanie_rekrutacyjne_app   latest    ac24ce7b322d   16 minutes ago      1.89GB
marcin_zadanie_rekrutacyjne                      latest    ac24ce7b322d   16 minutes ago      1.89GB
mr_probka                                        latest    9f12e5b2ab82   20 minutes ago      1.89GB
<none>                                           <none>    83979b2e67fb   22 minutes ago      1.89GB
<none>                                           <none>    3ce92221efd6   25 minutes ago      1.89GB
<none>                                           <none>    1af1d0ffd731   29 minutes ago      1.89GB
<none>                                           <none>    b1f9607c8d2f   30 minutes ago      1.89GB
<none>                                           <none>    5c37bbb18b25   32 minutes ago      1.89GB
<none>                                           <none>    435a69

Łącze się z Docker Hub...

In [18]:
!sudo docker login

[sudo] password for marcin: WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/marcin/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


...nadaję tag mojemu obrazowi...

In [19]:
!sudo docker tag marcin_zadanie_rekrutacyjne marcinrzadanie/marcin_zadanie_rekrutacyjne_app

[sudo] password for marcin: 

...i pushuje go na Docker Hub.

In [20]:
!sudo docker push marcinrzadanie/marcin_zadanie_rekrutacyjne_app

[sudo] password for marcin: Using default tag: latest
The push refers to repository [docker.io/marcinrzadanie/marcin_zadanie_rekrutacyjne_app]

6a5f20d9: Preparing 
444f4643: Preparing 
4e8f97f2: Preparing 
cf575ce9: Preparing 
18e51f11: Preparing 
1ba64cc9: Preparing 
5f8cdac6: Preparing 
a12a5f5c: Preparing 
6448b1d7: Preparing 
002ee6ca: Preparing 
dae211b4: Preparing 
af798ace: Preparing 
4fdbedc0: Preparing 
e810d54a: Preparing 


f575ce9: Pushing    239MB/1.002GBPushing  116.9MB/510.1MBf575ce9: Pushed   1.022GB/1.002GBlatest: digest: sha256:4f78b6d21f2303d93c12d8b9cc48c75c4d78e95fb4903f38d38459b7a55a73f2 size: 3472


Jest gotowy do pobrania i w dalszym kroku do wdrożenia.

## Kubernetes

Z uwagi na fakt, że buduję wszystko na moim laptopie, w tym przypadku decyduje się na użycie [minikube](https://minikube.sigs.k8s.io/docs/start/), który pozwoli ma postawienie Kubernetesa na środowisku lokalnym.

Uruchamiam go.

In [22]:
!minikube start

😄  minikube v1.22.0 on Ubuntu 18.04
✨  Using the virtualbox driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🏃  Updating the running virtualbox "minikube" VM ...
🐳  Preparing Kubernetes v1.21.2 on Docker 20.10.6 ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


Deploy'uje aplikacje...

In [40]:
!kubectl create deployment marcinrzadanie2 --image=marcinrzadanie/marcin_zadanie_rekrutacyjne_app:latest

deployment.apps/marcinrzadanie2 created


... "wystawiam ją na świat" na porcie 8000, dlatego, że aplikacja Flask "słucha" na porcie 8000.

In [41]:
!kubectl expose deployment marcinrzadanie2 --type=NodePort --port=8000

service/marcinrzadanie2 exposed


Sprawdzam czy jest...

In [42]:
!kubectl get services marcinrzadanie2

NAME              TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
marcinrzadanie2   NodePort   10.97.156.128   <none>        8000:32246/TCP   2s


Sprawdzam podstawowe informacje o aplikacji... Interesuje mnie URL.

*W tym momencie mógłbym użyc komendy: `kubectl port-forward service/marcinrzadanie2 8000:8080` i adres strony zamieni się wtedy na `http://localhost:8000/` jednakże zablokuje to niepotrzebne ten notebook, a będzie on nam jeszcze potrzebny w dalszych krokach. W takim wypadku korzystam z "gotowego" URLa.*



In [43]:
!minikube service marcinrzadanie2

|-----------|-----------------|-------------|-----------------------------|
| NAMESPACE |      NAME       | TARGET PORT |             URL             |
|-----------|-----------------|-------------|-----------------------------|
| default   | marcinrzadanie2 |        8000 | http://192.168.99.100:32246 |
|-----------|-----------------|-------------|-----------------------------|
🎉  Opening service default/marcinrzadanie2 in default browser...


Aplikacja jest postawiona, mamy jej adres. Spróbujmy odpytać ją dwoma przykładami.

In [44]:
!curl -X POST http://192.168.99.100:32246/predict -d '{"twitter": "Bardzo Cie lubię :)"}'

Tag: non-harmful, inference time: 0.0029 seconds

In [45]:
!curl -X POST http://192.168.99.100:32246/predict -d '{"twitter": "A Ciebie nie lubię !!"}'

Tag: non-harmful, inference time: 0.0018 seconds

Udało się :) API działa i jest gotowe do użycia.